In [ ]:
import geopandas as gpd
import rasterio
import tqdm.notebook as tqdm
from rasterio.windows import Window
from rasterio.windows import from_bounds
from rasterio.mask import mask
import numpy as np
import cv2

In [ ]:
#read first grid
grid=gpd.read_file("grid_shape/grid1.shp")

In [ ]:
#open all constant covariates for post earthquake data
dist_riv=rasterio.open("covariates/distance_river.tif")
PGA_mean=rasterio.open("covariates/PGA_Mean.tif")
PGA_std=rasterio.open("covariates/PGA_std.tif")
precipitation_max=rasterio.open("covariates/precipitation_max.tif")
precipitation_p95=rasterio.open("covariates/precipitation_p95.tif")
topographic_params=rasterio.open("covariates/Topographic_params.tif")

In [ ]:
#open area density difference of landslides
area_density=rasterio.open("inventory/landslide_positive_v2.tif")

In [ ]:
def clip_by_bound_label(polygon,raster,epoch=None):
    #print(polygon.bounds)
    a,b=(polygon.bounds[0] , polygon.bounds[1] )
    x,y=(polygon.bounds[0]+4000 , polygon.bounds[1]+4000 )
    row,col=raster.index(a, b)
    row_end,col_end=raster.index(x,y)
    
    if epoch==None:
        data_val=raster.read()
        data_sub=data_val[:,row_end:row,col:col_end]
        #print(data_val.shape,data_sub.shape)
        return data_sub
    else:
        data_val=np.expand_dims(raster.read(epoch),axis=0)
        data_sub=data_val[:,row_end:row,col:col_end]
        #print(data_val.shape,data_sub.shape)
        return data_sub
    #print(row,col,row_end,col_end) #209 175 205 179
def clip_by_bound_covar(polygon,raster,epoch=None):
    a,b=(polygon.bounds[0] , polygon.bounds[1] )
    x,y=(polygon.bounds[0]+4000 , polygon.bounds[1]+4000 )
    
    row,col=raster.index(a, b)
    row_end,col_end=raster.index(x,y)
    #row_end=row-128
    #col_end=col+128
    #print(row,col,row_end,col_end)
    #print(row,row_end,col,col_end)
    if epoch==None:
        data_val=raster.read()
        data_sub=data_val[:,row_end:row,col:col_end]
        return data_sub
    else:
        data_val=np.expand_dims(raster.read(epoch),axis=0)
        data_sub=data_val[:,row_end:row,col:col_end]
        return data_sub
    #print(row,col,row_end,col_end) #209 175 205 179

In [ ]:
# prepare data for post earthquake in 3 classess
computed_grid=[]
def clip_covars(polygon,epoch,monssons_after_eq):
    topo_data=clip_by_bound_covar(polygon,topographic_params)
    dist_riv_data=clip_by_bound_covar(polygon,dist_riv)#rasterio.mask.mask(dist_riv, [polygon], crop=True,nodata=0)
    PGA_mean_data=np.exp(clip_by_bound_covar(polygon,PGA_mean))#rasterio.mask.mask(PGA_mean, [polygon], crop=True,nodata=0)
    PGA_std_data=np.exp(clip_by_bound_covar(polygon,PGA_std))#rasterio.mask.mask(PGA_std, [polygon], crop=True,nodata=0)
    precipitation_max_data=clip_by_bound_covar(polygon,precipitation_max,epoch)#np.expand_dims(rasterio.mask.mask(precipitation_max, [polygon], crop=True,nodata=0)[0][epoch,:,:], axis=0)
    precipitation_p95_data=clip_by_bound_covar(polygon,precipitation_p95,epoch)#np.expand_dims(rasterio.mask.mask(precipitation_p95, [polygon], crop=True,nodata=0)[0][epoch,:,:], axis=0)
    time=np.full(precipitation_p95_data.shape, monssons_after_eq)
    #print(topo_data.shape)
    stacked_data=np.concatenate((topo_data, dist_riv_data,PGA_mean_data,PGA_std_data,precipitation_max_data,precipitation_p95_data,time ), axis=0)
    stacked_data=np.moveaxis(stacked_data, 0, -1).astype(np.float32)
    #print(stacked_data.shape)
    #print(time)
    return stacked_data
    #return arrray of 128x128

def clip_label(polygon,epoch):
    area_den=clip_by_bound_label(polygon,area_density,epoch)
    #print(rasterio.mask.mask(area_density, [polygon], nodata=0,all_touched=False,crop=True)[1])
    landslide_present=np.where(area_den >0, 1, 0)
    ones=np.ones(landslide_present.shape)
    landslide_absent=ones-landslide_present
    stacked_data=np.stack((landslide_present,landslide_absent,area_den),axis=0).astype(np.float32)
    #print(stacked_data)
    stacked_data=np.moveaxis(stacked_data, 0, -1)
    #print(stacked_data)
    return stacked_data
    #return array of 4x4
seond=True
monssons_dict={4:0,5:1,6:1,7:2,8:2,9:3,10:3,11:4}
first=True
i=0
for epoch in range(4,12):
    for ind,row in tqdm.tqdm(grid.iterrows(),total=grid.shape[0]):
        
        polygon=row.geometry
        #print(polygon)
        diff_area_epoch=epoch-1
        precipitation_epoch=epoch
        monsoons_after_eq=monssons_dict[epoch]
        label_temp=clip_label(polygon,diff_area_epoch)
        cov_temp=clip_covars(polygon,precipitation_epoch,monsoons_after_eq)

        label_data=np.expand_dims(label_temp, axis=0)
        cov_temp=np.expand_dims(cov_temp, axis=0)
        if first:
            if (label_temp.shape==(1, 4, 4, 3)) and (cov_temp.shape==(1,128,128,13)):
                label=label_temp
                covariate=cov_temp
                first=False
            else:
                print(f"error the shape has issues {label_temp.shape} and {cov_temp.shape}")
                computed_grid.append(ind)
                print(ind)
                pass

        else:
            #print(label.shape,label_temp.shape)
            if (label_temp.shape==(1, 4, 4, 3)) and (cov_temp.shape==(1,128,128,13)):
                label=np.concatenate((label, label_temp),axis=0)
                covariate=np.concatenate((covariate, cov_temp),axis=0)
            else:
                print(f"error the shape has issues {label_temp.shape} and {cov_temp.shape}")
                computed_grid.append(ind)
                print(ind)
                pass
        if len(computed_grid)>=8:
            np.save('error_grids.npy',np.array(computed_grid))
        
    # np.save(f'label_{str(epoch)}.npy', label)
    # np.save(f'covars_{str(epoch)}.npy', covariate)
    # print(f'shape is {label.shape, covariate.shape}')
    first=True
    print(label.shape,covariate.shape)
        

In [ ]:
#numpy save the data
print(label.shape,covariate.shape)

In [ ]:
indexes=np.where(label <0)
index_del=np.unique(indexes[0])
print(index_del)
ydata=np.delete(label, index_del,axis=0)
xdata=np.delete(covariate, index_del,axis=0)
np.save('TrainingLabelV2.npy', ydata)
np.save('TrainingCovarV2.npy', xdata)
